## Data Organization

- days since previous game(s)
-

Note that we'll need to do this per season per team


In [27]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [28]:
def haversine_np(lon1, lat1, lon2, lat2, km=True):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    if km:
        dist = 6367 * c
    else:
        dist = 3950 * c
    return dist

### Dataprep Start

In [29]:
# get columns that I want
# make two df's (one for home and one for away)
# stack them
# sort by team and date
# save out

In [30]:
# elevation data
elev_df = pd.read_csv("../data/city-elevation.csv")
print(elev_df.shape)
elev_df.head(2)


(29, 2)


,city,elevation
0,Denver,5191
1,Utah,4265


In [31]:

# load arena coordinates
coordinate_df = pd.read_csv("../data/arena_coordinates.csv")
coordinate_df["city_team_key"] = coordinate_df["City"] + " " + coordinate_df["Team"]
coordinate_df.head(2)


,Team,Arena,Latitude,Longitude,City,city_team_key
0,Hawks,State Farm Arena,33.757,84.396,Atlanta,Atlanta Hawks
1,Celtics,TD Garden,42.366,71.062,Boston,Boston Celtics


In [32]:
# load team info data
team_info_df = pd.read_csv("../data/team_info_with_timezone.csv")

team_info_df = (
    team_info_df
    # elevation
    .merge(
        elev_df,
        on="city",
        how="left",
    )
    # team location
    .merge(
        coordinate_df,
        left_on="full_name",
        right_on="city_team_key",
        how="left",
    )
)
print(team_info_df.shape)
team_info_df.head(10)

(30, 16)


,id,full_name,abbreviation,nickname,city,state,year_founded,timezone,time_diff_from_et,elevation,Team,Arena,Latitude,Longitude,City,city_team_key
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,America/New_York,0,1023,Hawks,State Farm Arena,33.757,84.396,Atlanta,Atlanta Hawks
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,America/New_York,0,10,Celtics,TD Garden,42.366,71.062,Boston,Boston Celtics
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,America/New_York,0,669,Cavaliers,Quicken Loans Arena,41.497,81.688,Cleveland,Cleveland Cavaliers
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,America/Chicago,-1,2,Pelicans,Smoothie King Center,29.949,90.082,New Orleans,New Orleans Pelicans
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,America/Chicago,-1,593,Bulls,United Center,41.881,87.675,Chicago,Chicago Bulls
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980,America/Chicago,-1,426,Mavericks,American Airlines Center,32.791,96.810,Dallas,Dallas Mavericks
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976,America/Denver,-2,5191,Nuggets,Pepsi Center,39.749,105.008,Denver,Denver Nuggets
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946,America/Los_Angeles,-3,19,Warriors,Oracle Arena,37.750,122.203,Golden State,Golden State Warriors
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967,America/Chicago,-1,45,Rockets,Toyota Center,29.751,95.362,Houston,Houston Rockets
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970,America/Los_Angeles,-3,239,Clippers,Staples Center,34.043,118.267,Los Angeles,Los Angeles Clippers


In [33]:
core_team_cols = [
    # "id",
    # "full_name",
    "city_team_key",
    "abbreviation",
    # "nickname",
    # "city",
    # "state",
    # "year_founded",
    "timezone",
    "time_diff_from_et",
    "elevation",
    # "Team",
    # "Arena",
    "Latitude",
    "Longitude",
    # "City",
]

In [34]:
# games_df = pd.read_csv("../data/br_all_games.csv")
games_df = pd.read_csv("../data/2014-2024_reg_season_raw.csv")

# games_df.drop(
#     [
#         "link",
#         # "Arena",
#     ],
#     axis=1,
#     inplace=True
# )


# this filter simplifies things by excluding franchise changes
# games_df = games_df[games_df['season_start_year'] >= 2014]
# games_df = games_df[games_df["season_start_year"] >= 2014]


# keep it at current season for simplicity
# games_df = games_df[games_df["season_start_year"] == 2023].copy()


print(games_df.shape)
games_df.head(2)

(23898, 9)


,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time
0,Atlanta Hawks,Toronto Raptors,Wed,2014-10-29,7:30p,2014,0,0,2014-10-29 19:30:00
1,Atlanta Hawks,Indiana Pacers,Sat,2014-11-01,7:30p,2014,0,1,2014-11-01 19:30:00


In [35]:
print(games_df['team'].nunique())
games_df['team'].unique()

30


array(['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets',
       'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons',
       'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
       'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies',
       'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves',
       'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder',
       'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns',
       'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs',
       'Toronto Raptors', 'Utah Jazz', 'Washington Wizards'], dtype=object)

In [36]:
print([x for x in games_df['team'].unique() if x not in team_info_df['full_name'].unique()])
print([x for x in team_info_df['full_name'].unique() if x not in games_df['team'].unique()])


[]
[]


In [37]:
# assert False

In [38]:
# join auxillary tables onto the main dataset (game df is the main)
df = (
    games_df
    # team info
    .merge(
        team_info_df[core_team_cols],
        left_on="team",
        right_on="city_team_key",
        how="left",
        suffixes=("", "_team")
    )
    # opponent info
    .merge(
        team_info_df[core_team_cols],
        left_on="opponent",
        right_on="city_team_key",
        how="left",
        suffixes=("", "_opponent")
    )
)
print(df.shape)
# df.info()
df.head()

(23898, 23)


,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time,city_team_key,abbreviation,timezone,time_diff_from_et,elevation,Latitude,Longitude,city_team_key_opponent,abbreviation_opponent,timezone_opponent,time_diff_from_et_opponent,elevation_opponent,Latitude_opponent,Longitude_opponent
0,Atlanta Hawks,Toronto Raptors,Wed,2014-10-29,7:30p,2014,0,0,2014-10-29 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Toronto Raptors,TOR,America/Toronto,0,260,43.644,79.379
1,Atlanta Hawks,Indiana Pacers,Sat,2014-11-01,7:30p,2014,0,1,2014-11-01 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Indiana Pacers,IND,America/New_York,0,707,39.764,86.156
2,Atlanta Hawks,San Antonio Spurs,Wed,2014-11-05,8:30p,2014,0,0,2014-11-05 20:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,San Antonio Spurs,SAS,America/Chicago,-1,641,29.427,98.438
3,Atlanta Hawks,Charlotte Hornets,Fri,2014-11-07,7:00p,2014,0,0,2014-11-07 19:00:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Charlotte Hornets,CHA,America/New_York,0,721,35.225,80.839
4,Atlanta Hawks,New York Knicks,Sat,2014-11-08,7:30p,2014,0,1,2014-11-08 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,New York Knicks,NYK,America/New_York,0,35,40.751,73.993


In [39]:
# convert to date type
df["game_time"] = pd.to_datetime(df["game_time"])
df["date"] = pd.to_datetime(df["date"])

# sort by team and game date (there were issues when sorted by game id for some reason)
df = df.sort_values(by=["abbreviation", "game_time"])

# mark covid games
df['is_covid_bubble'] = np.where((df['game_time'] >= "2020-07-09") & (df['game_time'] <= "2020-10-11"), 1, 0)

# mark pre-post TBD nba cup games
df['is_pre_nba_cup_knockout'] = np.where((df['game_time'] >= "2023-07-09") & (df['game_time'] <= "2023-12-02"), 1, 0)
df['is_post_nba_cup'] = np.where((df['game_time'] >= "2020-07-09") & (df['game_time'] >= "2023-12-11"), 1, 0)


# time diff
df['game_time_diff_from_et'] = np.where(df['is_home_game'], df['time_diff_from_et'], df["time_diff_from_et_opponent"])

# game coordinates
df["game_lat"] = np.where(
    df["is_home_game"], df["Latitude"], df["Latitude_opponent"]
)
df["game_lon"] = np.where(
    df["is_home_game"], df["Longitude"], df["Longitude_opponent"]
)

# this should be in for loop section to not cross seasons
# df["prev_game_lon"] = df["game_lon"].shift(1).fillna(df["Longitude"])
# df["prev_game_lat"] = df["game_lat"].shift(1).fillna(df["Latitude"])

# game elevation
df['game_elevation'] = np.where(df['is_home_game'], df['elevation'], df['elevation_opponent'])


# make a unique game key for each game
df['home_team_abbr'] = np.where(df['is_home_game'], df['abbreviation'], df['abbreviation_opponent'])
df['away_team_abbr'] = np.where(df['is_home_game'], df['abbreviation_opponent'], df['abbreviation'])
df['game_id'] = df['away_team_abbr'] + "@" + df['home_team_abbr'] + "_" + df['date'].astype(str)

print(df.shape)

# should be ??? number of covid games (includes playoffs)
print(f"covid_games: {df['is_covid_bubble'].sum()}")
# df.info()
df.head(7)

(23898, 33)
covid_games: 176


,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time,city_team_key,abbreviation,timezone,time_diff_from_et,elevation,Latitude,Longitude,city_team_key_opponent,abbreviation_opponent,timezone_opponent,time_diff_from_et_opponent,elevation_opponent,Latitude_opponent,Longitude_opponent,is_covid_bubble,is_pre_nba_cup_knockout,is_post_nba_cup,game_time_diff_from_et,game_lat,game_lon,game_elevation,home_team_abbr,away_team_abbr,game_id
0,Atlanta Hawks,Toronto Raptors,Wed,2014-10-29,7:30p,2014,0,0,2014-10-29 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Toronto Raptors,TOR,America/Toronto,0,260,43.644,79.379,0,0,0,0,43.644,79.379,260,TOR,ATL,ATL@TOR_2014-10-29
1,Atlanta Hawks,Indiana Pacers,Sat,2014-11-01,7:30p,2014,0,1,2014-11-01 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Indiana Pacers,IND,America/New_York,0,707,39.764,86.156,0,0,0,0,33.757,84.396,1023,ATL,IND,IND@ATL_2014-11-01
2,Atlanta Hawks,San Antonio Spurs,Wed,2014-11-05,8:30p,2014,0,0,2014-11-05 20:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,San Antonio Spurs,SAS,America/Chicago,-1,641,29.427,98.438,0,0,0,-1,29.427,98.438,641,SAS,ATL,ATL@SAS_2014-11-05
3,Atlanta Hawks,Charlotte Hornets,Fri,2014-11-07,7:00p,2014,0,0,2014-11-07 19:00:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Charlotte Hornets,CHA,America/New_York,0,721,35.225,80.839,0,0,0,0,35.225,80.839,721,CHA,ATL,ATL@CHA_2014-11-07
4,Atlanta Hawks,New York Knicks,Sat,2014-11-08,7:30p,2014,0,1,2014-11-08 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,New York Knicks,NYK,America/New_York,0,35,40.751,73.993,0,0,0,0,33.757,84.396,1023,ATL,NYK,NYK@ATL_2014-11-08
5,Atlanta Hawks,New York Knicks,Mon,2014-11-10,7:30p,2014,0,0,2014-11-10 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,New York Knicks,NYK,America/New_York,0,35,40.751,73.993,0,0,0,0,40.751,73.993,35,NYK,ATL,ATL@NYK_2014-11-10
6,Atlanta Hawks,Utah Jazz,Wed,2014-11-12,7:30p,2014,0,1,2014-11-12 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Utah Jazz,UTA,America/Denver,-2,4265,40.768,111.901,0,0,0,0,33.757,84.396,1023,ATL,UTA,UTA@ATL_2014-11-12


In [40]:
# df['game_time_diff_from_et'].diff()


In [41]:
# # sanity checks
# temp = df[
#     (df["is_covid_bubble"] == 1)
#     # & (df['team'] == "Utah Jazz")
#     ].sort_values(by="game_time")
# print(temp.shape)
# temp.tail(20)

In [42]:
# pause
# assert False

In [43]:
team_datasets = []
for team in df["team"].unique():
    team_df = df[df["team"] == team].copy()
    for season in team_df["season_start_year"].unique():
        # print(f"{team}: {season}")
        team_season_df = team_df[team_df["season_start_year"] == season].copy()
        
        # # mark season game ids
        # team_season_df["team_game_id"] = team_season_df.groupby(["abbreviation"]).cumcount() + 1

        
        team_season_df["time_since_prev_game"] = (
            team_season_df["game_time"].diff().dt.total_seconds() / 3600
        ).fillna(-1)

        team_season_df["is_last_game_home"] = (
            team_season_df["is_home_game"].shift(1).fillna(1).astype(int)
        )
        team_season_df["is_2_games_ago_home"] = (
            team_season_df["is_home_game"].shift(2).fillna(1).astype(int)
        )
        team_season_df["is_3_games_ago_home"] = (
            team_season_df["is_home_game"].shift(3).fillna(1).astype(int)
        )
        team_season_df["is_4_games_ago_home"] = (
            team_season_df["is_home_game"].shift(4).fillna(1).astype(int)
        )
        team_season_df["is_5_games_ago_home"] = (
            team_season_df["is_home_game"].shift(5).fillna(1).astype(int)
        )
        team_season_df["is_6_games_ago_home"] = (
            team_season_df["is_home_game"].shift(6).fillna(1).astype(int)
        )
        team_season_df["is_7_games_ago_home"] = (
            team_season_df["is_home_game"].shift(7).fillna(1).astype(int)
        )

        # num away games in last 5
        team_season_df["num_away_last_5_games"] = team_season_df[
            [
                "is_last_game_home",
                "is_2_games_ago_home",
                "is_3_games_ago_home",
                "is_4_games_ago_home",
                "is_5_games_ago_home",
            ]
        ].apply(lambda row: (row == 0).sum(), axis=1)

        # streak of home vs away games
        team_season_df["start_of_streak"] = team_season_df["is_home_game"].ne(
            team_season_df["is_home_game"].shift()
        )
        team_season_df["streak_id"] = team_season_df["start_of_streak"].cumsum()
        team_season_df["streak_counter"] = (
            team_season_df.groupby("streak_id").cumcount() + 1
        )
        # count of previous consecutive away games
        team_season_df["incoming_away_game_streak"] = np.where(
            team_season_df["is_last_game_home"] == False,
            team_season_df["streak_counter"] - 1,
            0,
        )

        team_season_df["days_since_1_games_ago"] = (
            team_season_df["date"] - team_season_df["date"].shift(1)
        ).dt.days.fillna(150)
        team_season_df["days_since_2_games_ago"] = (
            team_season_df["date"] - team_season_df["date"].shift(2)
        ).dt.days.fillna(150)
        team_season_df["days_since_3_games_ago"] = (
            team_season_df["date"] - team_season_df["date"].shift(3)
        ).dt.days.fillna(150)
        team_season_df["days_since_4_games_ago"] = (
            team_season_df["date"] - team_season_df["date"].shift(4)
        ).dt.days.fillna(150)
        team_season_df["days_since_5_games_ago"] = (
            team_season_df["date"] - team_season_df["date"].shift(5)
        ).dt.days.fillna(150)
        team_season_df["days_since_6_games_ago"] = (
            team_season_df["date"] - team_season_df["date"].shift(6)
        ).dt.days.fillna(150)
        team_season_df["days_since_7_games_ago"] = (
            team_season_df["date"] - team_season_df["date"].shift(7)
        ).dt.days.fillna(150)

        team_season_df["num_games_last_4_days"] = team_season_df[
            [
                "days_since_1_games_ago",
                "days_since_2_games_ago",
                "days_since_3_games_ago",
                "days_since_4_games_ago",
                "days_since_5_games_ago",
                "days_since_6_games_ago",
                "days_since_7_games_ago",
            ]
        ].apply(lambda row: ((row <= 4) & (row > 0)).sum(), axis=1)
        team_season_df["num_games_last_7_days"] = team_season_df[
            [
                "days_since_1_games_ago",
                "days_since_2_games_ago",
                "days_since_3_games_ago",
                "days_since_4_games_ago",
                "days_since_5_games_ago",
                "days_since_6_games_ago",
                "days_since_7_games_ago",
            ]
        ].apply(lambda row: ((row <= 7) & (row > 0)).sum(), axis=1)

        # distance from previous game
        
        
        team_season_df["prev_game_lon"] = team_season_df["game_lon"].shift(1).fillna(team_season_df["Longitude"])
        team_season_df["prev_game_lat"] = team_season_df["game_lat"].shift(1).fillna(team_season_df["Latitude"])
        
        team_season_df["distance_from_previous_game"] = haversine_np(
            team_season_df["game_lon"],
            team_season_df["game_lat"],
            team_season_df["prev_game_lon"],
            team_season_df["prev_game_lat"],
            km=False,
        )

        # distance traveled up to this game --------------------
        # Create a list to store the cumulative distances
        cumulative_distance = 0
        cumulative_distances = []
        # Iterate through the DataFrame rows
        for index, row in team_season_df.iterrows():
            # Check if it's a home game
            if row["is_last_game_home"] == 1:
                # print("here", index)
                cumulative_distance = 0  # Reset cumulative distance

            # Add the current distance to cumulative_distance
            cumulative_distance += row["distance_from_previous_game"]

            # Append the cumulative distance to the list
            cumulative_distances.append(cumulative_distance)

        # Add the cumulative_distances list as a new column to the DataFrame
        team_season_df["cumulative_distance"] = cumulative_distances
        # ------------------------------------------------------------

        # concatenate
        team_datasets.append(team_season_df)

processed_df = pd.concat(team_datasets).reset_index(drop=True)
print(processed_df.shape)
processed_df.head(2)

(23898, 59)


,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time,city_team_key,abbreviation,timezone,time_diff_from_et,elevation,Latitude,Longitude,city_team_key_opponent,abbreviation_opponent,timezone_opponent,time_diff_from_et_opponent,elevation_opponent,Latitude_opponent,Longitude_opponent,is_covid_bubble,is_pre_nba_cup_knockout,is_post_nba_cup,game_time_diff_from_et,game_lat,game_lon,game_elevation,home_team_abbr,away_team_abbr,game_id,time_since_prev_game,is_last_game_home,is_2_games_ago_home,is_3_games_ago_home,is_4_games_ago_home,is_5_games_ago_home,is_6_games_ago_home,is_7_games_ago_home,num_away_last_5_games,start_of_streak,streak_id,streak_counter,incoming_away_game_streak,days_since_1_games_ago,days_since_2_games_ago,days_since_3_games_ago,days_since_4_games_ago,days_since_5_games_ago,days_since_6_games_ago,days_since_7_games_ago,num_games_last_4_days,num_games_last_7_days,prev_game_lon,prev_game_lat,distance_from_previous_game,cumulative_distance
0,Atlanta Hawks,Toronto Raptors,Wed,2014-10-29,7:30p,2014,0,0,2014-10-29 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Toronto Raptors,TOR,America/Toronto,0,260,43.644,79.379,0,0,0,0,43.644,79.379,260,TOR,ATL,ATL@TOR_2014-10-29,-1.0,1,1,1,1,1,1,1,0,True,1,1,0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,0,0,84.396,33.757,732.742992,732.742992
1,Atlanta Hawks,Indiana Pacers,Sat,2014-11-01,7:30p,2014,0,1,2014-11-01 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Indiana Pacers,IND,America/New_York,0,707,39.764,86.156,0,0,0,0,33.757,84.396,1023,ATL,IND,IND@ATL_2014-11-01,72.0,0,1,1,1,1,1,1,1,True,2,1,0,3.0,150.0,150.0,150.0,150.0,150.0,150.0,1,1,79.379,43.644,732.742992,1465.485985


In [44]:
# other fields I can add

# is back-to-back
processed_df['is_back_to_back'] = np.where(processed_df['days_since_1_games_ago'] <= 1, 1, 0)

processed_df.head(2)



,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time,city_team_key,abbreviation,timezone,time_diff_from_et,elevation,Latitude,Longitude,city_team_key_opponent,abbreviation_opponent,timezone_opponent,time_diff_from_et_opponent,elevation_opponent,Latitude_opponent,Longitude_opponent,is_covid_bubble,is_pre_nba_cup_knockout,is_post_nba_cup,game_time_diff_from_et,game_lat,game_lon,game_elevation,home_team_abbr,away_team_abbr,game_id,time_since_prev_game,is_last_game_home,is_2_games_ago_home,is_3_games_ago_home,is_4_games_ago_home,is_5_games_ago_home,is_6_games_ago_home,is_7_games_ago_home,num_away_last_5_games,start_of_streak,streak_id,streak_counter,incoming_away_game_streak,days_since_1_games_ago,days_since_2_games_ago,days_since_3_games_ago,days_since_4_games_ago,days_since_5_games_ago,days_since_6_games_ago,days_since_7_games_ago,num_games_last_4_days,num_games_last_7_days,prev_game_lon,prev_game_lat,distance_from_previous_game,cumulative_distance,is_back_to_back
0,Atlanta Hawks,Toronto Raptors,Wed,2014-10-29,7:30p,2014,0,0,2014-10-29 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Toronto Raptors,TOR,America/Toronto,0,260,43.644,79.379,0,0,0,0,43.644,79.379,260,TOR,ATL,ATL@TOR_2014-10-29,-1.0,1,1,1,1,1,1,1,0,True,1,1,0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,0,0,84.396,33.757,732.742992,732.742992,0
1,Atlanta Hawks,Indiana Pacers,Sat,2014-11-01,7:30p,2014,0,1,2014-11-01 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Indiana Pacers,IND,America/New_York,0,707,39.764,86.156,0,0,0,0,33.757,84.396,1023,ATL,IND,IND@ATL_2014-11-01,72.0,0,1,1,1,1,1,1,1,True,2,1,0,3.0,150.0,150.0,150.0,150.0,150.0,150.0,1,1,79.379,43.644,732.742992,1465.485985,0


In [45]:
# # how many quick turn around games are there in 2023
# a = processed_df[
#     (processed_df["time_since_prev_game"] <= 24)
#     & (processed_df["season_start_year"].isin(list(range(2014, 2024))))
# ]

# a.pivot_table(
#     index=[
#         "season_start_year",
#         # "abbreviation",
#     ],
#     values=["game_id"],
#     aggfunc=["count"],
# )/2

In [46]:
processed_df.columns

Index(['team', 'opponent', 'day_of_week', 'date', 'Start (ET)',
       'season_start_year', 'is_playoff', 'is_home_game', 'game_time',
       'city_team_key', 'abbreviation', 'timezone', 'time_diff_from_et',
       'elevation', 'Latitude', 'Longitude', 'city_team_key_opponent',
       'abbreviation_opponent', 'timezone_opponent',
       'time_diff_from_et_opponent', 'elevation_opponent', 'Latitude_opponent',
       'Longitude_opponent', 'is_covid_bubble', 'is_pre_nba_cup_knockout',
       'is_post_nba_cup', 'game_time_diff_from_et', 'game_lat', 'game_lon',
       'game_elevation', 'home_team_abbr', 'away_team_abbr', 'game_id',
       'time_since_prev_game', 'is_last_game_home', 'is_2_games_ago_home',
       'is_3_games_ago_home', 'is_4_games_ago_home', 'is_5_games_ago_home',
       'is_6_games_ago_home', 'is_7_games_ago_home', 'num_away_last_5_games',
       'start_of_streak', 'streak_id', 'streak_counter',
       'incoming_away_game_streak', 'days_since_1_games_ago',
       'days_sinc

In [47]:
self_join_keeps = [
    "team",
    "game_id",
    # "opponent",
    # "day_of_week",
    # "date",
    # "Start (ET)",
    # "season_start_year",
    # "is_playoff",
    # "is_home_game",
    # "game_time",
    # "city_team_key",
    "abbreviation",
    # "nickname",
    # "timezone",
    # "time_diff_from_et",
    # "elevation",
    # "Latitude",
    # "Longitude",
    # "city_team_key_opponent",
    # "abbreviation_opponent",
    # "nickname_opponent",
    # "timezone_opponent",
    # "time_diff_from_et_opponent",
    # "elevation_opponent",
    # "Latitude_opponent",
    # "Longitude_opponent",
    # "is_covid_bubble",
    # "game_timediff_from_et",
    # "game_lat",
    # "game_lon",
    # "prev_game_lon",
    # "prev_game_lat",
    # "home_team_abbr",
    # "away_team_abbr",
    "time_since_prev_game",
    # "is_last_game_home",
    # "is_2_games_ago_home",
    # "is_3_games_ago_home",
    # "is_4_games_ago_home",
    # "is_5_games_ago_home",
    # "is_6_games_ago_home",
    # "is_7_games_ago_home",
    # "num_away_last_5_games",
    # "start_of_streak",
    # "streak_id",
    # "streak_counter",
    # "incoming_away_game_streak",
    # "days_since_1_games_ago",
    # "days_since_2_games_ago",
    # "days_since_3_games_ago",
    # "days_since_4_games_ago",
    # "days_since_5_games_ago",
    # "days_since_6_games_ago",
    # "days_since_7_games_ago",
    # "num_games_last_4_days",
    "num_games_last_7_days",
    "distance_from_previous_game",
    "cumulative_distance",
]
self_join_keeps

['team',
 'game_id',
 'abbreviation',
 'time_since_prev_game',
 'num_games_last_7_days',
 'distance_from_previous_game',
 'cumulative_distance']

In [48]:
new_df = pd.merge(
    processed_df,
    processed_df[self_join_keeps],
    left_on=["game_id", "abbreviation_opponent"],
    right_on=[
        "game_id",
        "abbreviation",
    ],
    suffixes=("", "_opponent"),
)#.drop("team_abbreviation_opponent", axis=1)
print(new_df.shape)
new_df.head(2)

(23898, 66)


,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time,city_team_key,abbreviation,timezone,time_diff_from_et,elevation,Latitude,Longitude,city_team_key_opponent,abbreviation_opponent,timezone_opponent,time_diff_from_et_opponent,elevation_opponent,Latitude_opponent,Longitude_opponent,is_covid_bubble,is_pre_nba_cup_knockout,is_post_nba_cup,game_time_diff_from_et,game_lat,game_lon,game_elevation,home_team_abbr,away_team_abbr,game_id,time_since_prev_game,is_last_game_home,is_2_games_ago_home,is_3_games_ago_home,is_4_games_ago_home,is_5_games_ago_home,is_6_games_ago_home,is_7_games_ago_home,num_away_last_5_games,start_of_streak,streak_id,streak_counter,incoming_away_game_streak,days_since_1_games_ago,days_since_2_games_ago,days_since_3_games_ago,days_since_4_games_ago,days_since_5_games_ago,days_since_6_games_ago,days_since_7_games_ago,num_games_last_4_days,num_games_last_7_days,prev_game_lon,prev_game_lat,distance_from_previous_game,cumulative_distance,is_back_to_back,team_opponent,abbreviation_opponent,time_since_prev_game_opponent,num_games_last_7_days_opponent,distance_from_previous_game_opponent,cumulative_distance_opponent
0,Atlanta Hawks,Toronto Raptors,Wed,2014-10-29,7:30p,2014,0,0,2014-10-29 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Toronto Raptors,TOR,America/Toronto,0,260,43.644,79.379,0,0,0,0,43.644,79.379,260,TOR,ATL,ATL@TOR_2014-10-29,-1.0,1,1,1,1,1,1,1,0,True,1,1,0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,0,0,84.396,33.757,732.742992,732.742992,0,Toronto Raptors,TOR,-1.0,0,0.000000,0.000000
1,Atlanta Hawks,Indiana Pacers,Sat,2014-11-01,7:30p,2014,0,1,2014-11-01 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Indiana Pacers,IND,America/New_York,0,707,39.764,86.156,0,0,0,0,33.757,84.396,1023,ATL,IND,IND@ATL_2014-11-01,72.0,0,1,1,1,1,1,1,1,True,2,1,0,3.0,150.0,150.0,150.0,150.0,150.0,150.0,1,1,79.379,43.644,732.742992,1465.485985,0,Indiana Pacers,IND,24.5,2,425.353749,425.353749


In [49]:

new_df = new_df.assign(
    hours_of_rest_difference=new_df["time_since_prev_game"]
    - new_df["time_since_prev_game_opponent"],
    games_last_7_diff=new_df["num_games_last_7_days"]
    - new_df["num_games_last_7_days_opponent"],
    prev_game_distance_diff = new_df['distance_from_previous_game'] - new_df['distance_from_previous_game_opponent'],
    cumulative_distance_diff = new_df['cumulative_distance'] - new_df['cumulative_distance_opponent'],
)
# new_df["target"] = np.where(new_df["WL"] == "L", 1, 0)
print(processed_df.shape)
print(new_df.shape)
new_df.head()

(23898, 60)
(23898, 70)


,team,opponent,day_of_week,date,Start (ET),season_start_year,is_playoff,is_home_game,game_time,city_team_key,abbreviation,timezone,time_diff_from_et,elevation,Latitude,Longitude,city_team_key_opponent,abbreviation_opponent,timezone_opponent,time_diff_from_et_opponent,elevation_opponent,Latitude_opponent,Longitude_opponent,is_covid_bubble,is_pre_nba_cup_knockout,is_post_nba_cup,game_time_diff_from_et,game_lat,game_lon,game_elevation,home_team_abbr,away_team_abbr,game_id,time_since_prev_game,is_last_game_home,is_2_games_ago_home,is_3_games_ago_home,is_4_games_ago_home,is_5_games_ago_home,is_6_games_ago_home,is_7_games_ago_home,num_away_last_5_games,start_of_streak,streak_id,streak_counter,incoming_away_game_streak,days_since_1_games_ago,days_since_2_games_ago,days_since_3_games_ago,days_since_4_games_ago,days_since_5_games_ago,days_since_6_games_ago,days_since_7_games_ago,num_games_last_4_days,num_games_last_7_days,prev_game_lon,prev_game_lat,distance_from_previous_game,cumulative_distance,is_back_to_back,team_opponent,abbreviation_opponent,time_since_prev_game_opponent,num_games_last_7_days_opponent,distance_from_previous_game_opponent,cumulative_distance_opponent,hours_of_rest_difference,games_last_7_diff,prev_game_distance_diff,cumulative_distance_diff
0,Atlanta Hawks,Toronto Raptors,Wed,2014-10-29,7:30p,2014,0,0,2014-10-29 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Toronto Raptors,TOR,America/Toronto,0,260,43.644,79.379,0,0,0,0,43.644,79.379,260,TOR,ATL,ATL@TOR_2014-10-29,-1.0,1,1,1,1,1,1,1,0,True,1,1,0,150.0,150.0,150.0,150.0,150.0,150.0,150.0,0,0,84.396,33.757,732.742992,732.742992,0,Toronto Raptors,TOR,-1.0,0,0.000000,0.000000,0.0,0,732.742992,732.742992
1,Atlanta Hawks,Indiana Pacers,Sat,2014-11-01,7:30p,2014,0,1,2014-11-01 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Indiana Pacers,IND,America/New_York,0,707,39.764,86.156,0,0,0,0,33.757,84.396,1023,ATL,IND,IND@ATL_2014-11-01,72.0,0,1,1,1,1,1,1,1,True,2,1,0,3.0,150.0,150.0,150.0,150.0,150.0,150.0,1,1,79.379,43.644,732.742992,1465.485985,0,Indiana Pacers,IND,24.5,2,425.353749,425.353749,47.5,-1,307.389244,1040.132236
2,Atlanta Hawks,San Antonio Spurs,Wed,2014-11-05,8:30p,2014,0,0,2014-11-05 20:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,San Antonio Spurs,SAS,America/Chicago,-1,641,29.427,98.438,0,0,0,-1,29.427,98.438,641,SAS,ATL,ATL@SAS_2014-11-05,97.0,1,0,1,1,1,1,1,1,True,3,1,0,4.0,7.0,150.0,150.0,150.0,150.0,150.0,1,2,84.396,33.757,876.035741,876.035741,0,San Antonio Spurs,SAS,118.5,1,847.609727,1695.219453,-21.5,1,28.426015,-819.183712
3,Atlanta Hawks,Charlotte Hornets,Fri,2014-11-07,7:00p,2014,0,0,2014-11-07 19:00:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,Charlotte Hornets,CHA,America/New_York,0,721,35.225,80.839,0,0,0,0,35.225,80.839,721,CHA,ATL,ATL@CHA_2014-11-07,46.5,0,1,0,1,1,1,1,2,False,3,2,1,2.0,6.0,9.0,150.0,150.0,150.0,150.0,1,2,98.438,29.427,1098.431880,1974.467621,0,Charlotte Hornets,CHA,48.0,4,0.000000,0.000000,-1.5,-2,1098.431880,1974.467621
4,Atlanta Hawks,New York Knicks,Sat,2014-11-08,7:30p,2014,0,1,2014-11-08 19:30:00,Atlanta Hawks,ATL,America/New_York,0,1023,33.757,84.396,New York Knicks,NYK,America/New_York,0,35,40.751,73.993,0,0,0,0,33.757,84.396,1023,ATL,NYK,NYK@ATL_2014-11-08,24.5,0,0,1,0,1,1,1,3,True,4,1,0,1.0,3.0,7.0,10.0,150.0,150.0,150.0,2,3,80.839,35.225,226.016193,2200.483814,1,New York Knicks,NYK,24.0,4,744.221391,1706.426316,0.5,-1,-518.205198,494.057499


In [50]:
assert False

AssertionError: 

In [51]:
# 2023 reg season
temp = new_df[
    new_df['season_start_year'] == 2023
]
temp.to_csv("../data/2023-24_reg_season_clean.csv", index=False)
print(temp.shape)

(2400, 70)


In [52]:
# 2022 regular season
temp = new_df[
    new_df['season_start_year'] == 2022
]
temp.to_csv("../data/2022-23_reg_season_clean.csv", index=False)
print(temp.shape)

(2460, 70)


In [ ]:
assert False

In [ ]:
processed_df.columns

In [ ]:
new_df.to_csv("../data/game_data_clean.csv", index=False)

In [ ]:
# processed_df.to_csv("../data/game_data_1.csv", index=False)

In [ ]:
# assert False

In [ ]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(new_df, title="Profiling Report")


In [ ]:
# profile

In [ ]:
assert False

### Scratch
